In [41]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import pandas as pd

#from previous 2-30 code
import matplotlib.pyplot as plt
import numpy as np

from numpy.ma.core import var
from scipy.stats import t
from scipy.stats import f


# Importing statsmodels library for ANOVA
import statsmodels.api as sm
from statsmodels.formula.api import ols

#dataset (df)
url='https://raw.githubusercontent.com/ramirezramiro/linear-reg/main/Multiple%20Linear%20Reg%20(ch.3)/data(ch.3)/table-b3.csv'
df=pd.read_csv(url)

specific_columns=df[["y","x1","x6"]]
print(specific_columns.head())

# Extract the features (X) and target variable (y)
X = specific_columns[["x1","x6"]].values
y = specific_columns["y"].values

n_samples = len(y)
print("\nn_samples: ", n_samples)

n_features=len(specific_columns.columns)-1
print("\nn_features: ", n_features)

# Fit a linear regression model
model = LinearRegression().fit(X, y)
# Make predictions on the training data
y_pred = model.predict(X)

print("\n-----------")
print("a. Fit a Multiple Linear Regression model relating gasoline mileage y (miles per gallon) to engine displacement x1 and the number of carburator valves x6")
# Print the mathematical function of the model
print("y =", model.intercept_, "+", " + ".join(["{}*{}".format(coeff, feat) for coeff, feat in zip(model.coef_, ["x1", "x6"])]))
print("-----------\n")


print("\n-----------")
print("b. Construct the analysis-of-variance table and test for significance of regression.")
# Print the mathematical function of the model
model_anova = ols("y ~ x1 + x6", data=specific_columns).fit()
# Print the model summary
print(model_anova.summary())


print("\n-----------")
print("c. Calculate R_sq and R_sq_adj for this model. Compare this to the R_sq and R_sq_adj for the Simple Linear Regression model relating mileage y to engine displacement x1 in Problem 2.4.")
# Evaluate the model using R-squared
r2 = r2_score(y, y_pred)
print(" ")
print("R-squared:", r2)

def adjusted_r2_score(r2, n_samples, n_features):
    adj_r2 = 1 - ((1 - r2) * (n_samples - 1) / (n_samples - n_features - 1))
    return adj_r2

adj_r2 = adjusted_r2_score(r2, n_samples, n_features)
print("Adjusted R-squared:", adj_r2)

print("\nCalculate a new simple linear regression (sr) with y and x1")
# Extract the features (X) and target variable (y) in simple regression
X_sr = specific_columns["x1"].values
y_sr = specific_columns["y"].values

# Fit a linear regression model
model_sr = LinearRegression().fit(X_sr.reshape(-1, 1), y_sr)
# Make predictions on the training data
y_pred_sr = model_sr.predict(X_sr.reshape(-1, 1))

# Evaluate the sr model using R-squared
r2_sr = r2_score(y_sr, y_pred_sr)
print(" ")
print("R-squared sr:", r2_sr)

# Decrease in 1 the number of features, due to just using x1 and y
adj_r2_sr = adjusted_r2_score(r2, n_samples, n_features-1)
print("Adjusted R-squared sr:", adj_r2_sr)


print("\nIn this case, when adding x6, we can observe an increase of", r2-r2_sr, "in the r-square value" )
print("\n-----------")

print("d. Find a 95% CI for β1")
# calculate the confidence interval for x1
X1 = specific_columns['x1']
X1 = sm.add_constant(X1) # add constant term
model_ci = sm.OLS(y, X1).fit()
ci = model_ci.conf_int(alpha=0.05) # 95% confidence interval
print('\n95% confidence interval for x1: [{}, {}]'.format(ci[1][0], ci[1][1]))
print("\n-----------")

print("e. Compute the t statistics for testing H0:β1 = 0 and H0:β6 = 0. What conclusions can you draw?")
# Compute the t-statistic for x1 and x6
x_coef = model.coef_
x_std_err = np.sqrt(np.diag(np.linalg.inv(X.T @ X)))  # standard error of x1 and x6
x_t = x_coef / x_std_err

# Compute the p-value for x1 and x6
n_samples = X.shape[0]
n_features = X.shape[1]
df = n_samples - n_features - 1  # degrees of freedom
p_value_x1 = 2 * t.cdf(-abs(x_t[0]), df=df)  # two-tailed test
p_value_x6 = 2 * t.cdf(-abs(x_t[1]), df=df)  # two-tailed test

# Print the results
print("\nx1 coefficient:", x_coef[0])
print("x1 standard error:", x_std_err[0])
print("x1 t-statistic:", x_t[0])
print("x1 p-value:", p_value_x1)
if p_value_x1 < 0.05:
    print("Reject the null hypothesis that x1=0")
else:
    print("Fail to reject the null hypothesis that x1=0")
print(" ")
print("\nx6 coefficient:", x_coef[1])
print("x6 standard error:", x_std_err[1])
print("x6 t-statistic:", x_t[1])
print("x6 p-value:", p_value_x6)
if p_value_x6 < 0.05:
    print("Reject the null hypothesis that x6=0")
else:
    print("Fail to reject the null hypothesis that x6=0")
print("According to the t-statistic value of x1 and x6, both values are significant to the regression model")

print("\nIn addition, the F-statistic test was conducted as well")
# Extract the F-statistic and degrees of freedom from the ANOVA table
f_statistic = model_anova.fvalue
print("\nf_statistic: ", f_statistic)

df_reg = model_anova.df_model
df_res = model_anova.df_resid

# Compute the p-value for the F-statistic
p_value = f.sf(f_statistic, df_reg, df_res)
print("p-value for F-statistic:", p_value, "\n")
if p_value < 0.05:
    print("Reject the null hypothesis that all coefficients are equal to zero (p_value < 0.05)")
else:
    print("Fail to reject the null hypothesis that all coefficients are equal to zero (p_value > 0.05)")
print("\n-----------")

print("f. Find a 95% CI on the mean gasoline mileage when x1 = 275 cubic_in and x6 = 2 barrels.")
#input values of x1 and x6 for prediction
x_pred = np.array([275, 2]).reshape(1, -1)

# predict the mean y value and calculate the CI
y_pred_mean = model.predict(x_pred)[0]
n = n_samples  # number of observations
k = n_features  # number of predictors
alpha = 0.05  # significance level
df_error = n - k - 1  # degrees of freedom for error
t_crit = t.ppf(1 - alpha / 2, df_error)  # critical t-value
se = np.sqrt(np.sum((y - y_pred)**2) / df_error)  # standard error
margin_error = t_crit * se  # margin of error
CI_lower = y_pred_mean - margin_error  # lower bound of CI
CI_upper = y_pred_mean + margin_error  # upper bound of CI

# print the results
print("\n95% CI on the mean y when x1 = 275 and x6 = 2: [{:.2f}, {:.2f}]".format(CI_lower, CI_upper))
print("\n-----------")

print("\ng. Find a 95% prediction interval for a new observation on gasoline mileage when x 1 = 257 cubic_in and x6 = 2 barrels.")
# Make a new prediction
x_new = np.array([[257, 2]])
y_new_pred = model.predict(x_new)

# Calculate the 95% confidence interval for the prediction
t_value = t.ppf(1 - alpha / 2, n_samples - n_features - 1)
sigma = np.sqrt(np.sum((y - y_pred) ** 2) / (n_samples - n_features - 1))
CI_lower = y_new_pred - t_value * sigma * np.sqrt(1 + np.dot(np.dot(x_new, np.linalg.inv(np.dot(X.T, X))), x_new.T))
CI_upper = y_new_pred + t_value * sigma * np.sqrt(1 + np.dot(np.dot(x_new, np.linalg.inv(np.dot(X.T, X))), x_new.T))

print("\n95% New Confidence Interval when x1=257 and x6=2: {}, {}".format(CI_lower[0], CI_upper[0]))


       y     x1  x6
0  18.90  350.0   4
1  17.00  350.0   4
2  20.00  250.0   1
3  18.25  351.0   2
4  20.07  225.0   1

n_samples:  32

n_features:  2

-----------
a. Fit a Multiple Linear Regression model relating gasoline mileage y (miles per gallon) to engine displacement x1 and the number of carburator valves x6
y = 32.91004099671514 + -0.053024758490967085*x1 + 0.9294996701170247*x6
-----------


-----------
b. Construct the analysis-of-variance table and test for significance of regression.
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.786
Model:                            OLS   Adj. R-squared:                  0.771
Method:                 Least Squares   F-statistic:                     53.31
Date:                Sun, 07 May 2023   Prob (F-statistic):           1.93e-10
Time:                        16:36:57   Log-Likelihood:                -79.209
No. Observations:      